In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io as sio
import mne
import pandas as pd

In [9]:
# Function For Calculating PSD from a trial such as "jl_eeg3" 
# Return a 1D array of all the channel's Band's PSD

def calculate_psd_features(eeg_trial_data):
    # Create MNE Raw object
    info = mne.create_info(ch_names=[f'Channel {i + 1}' for i in range(eeg_trial_data.shape[0])], sfreq=235, ch_types='eeg')
    raw = mne.io.RawArray(eeg_trial_data, info)

    # Define frequency bands
    freq_bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 14), 'beta': (14, 30), 'gamma': (30, 50)}

    # Calculate PSD Features using Welch method
    psds, freqs = mne.time_frequency.psd_array_welch(raw.get_data(), sfreq=raw.info['sfreq'], fmin=0.5, fmax=50, n_jobs=1)

    # Organize PSD Features into DataFrame
    psd_data = []

    # Iterate through frequency bands
    for band, (fmin, fmax) in freq_bands.items():
        band_data = []

        # Iterate through channels
        for channel_idx, channel_name in enumerate(info.ch_names):
            freq_mask = (freqs >= fmin) & (freqs <= fmax)
            avg_psd = np.mean(psds[channel_idx, freq_mask])
            band_data.append(avg_psd)

        # Append channel's PSD features to the data list
        psd_data.extend(band_data)

    return psd_data

# Example usage:
# psd_data = calculate_psd_features(eeg_trial_data)
# print(psd_data)


In [5]:
# Calculate all the mat file's all trial's PSD 
# Return DF containing trial number and each trial's all channel's PSD

def process_subject_mat_files(mat_folder):
    freq_bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 14), 'beta': (14, 30), 'gamma': (30, 50)}
    subject_psd_data = []

    # Iterate through mat files in the folder
    for filename in os.listdir(mat_folder):
        if filename.endswith(".mat"):
            mat_file = os.path.join(mat_folder, filename)

            # Load the mat file
            mat_content = sio.loadmat(mat_file)

            # Identify trials in the mat file
            trial_names = [key for key in mat_content.keys() if 'eeg' in key.lower()]
            trial_names.sort()
            # Iterate through trials
            for trial_name in trial_names:
                eeg_trial_data = mat_content[trial_name]
                psd_features = calculate_psd_features(eeg_trial_data)
                subject_psd_data.append(psd_features)
    
    column_names = [f'channel_{i}_{band}' for i in range(62) for band in freq_bands.keys()]
    df = pd.DataFrame(data=np.array(subject_psd_data), columns=column_names)

    
    return df



In [10]:
mat_folder = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/Practice with datasets/Seed/Preprocessed_EEG"
subject_psd_data = process_subject_mat_files(mat_folder)

Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   200.000 secs
Ready.
Effective window size : 1.089 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47401
    Range : 0 ... 47400 =      0.000 ...   201.702 secs
Ready.
Effective window size : 1.089 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   200.000 secs
Ready.
Effective window size : 1.089 (s)
Creating RawArray with float64 data, n_channels=62, n_times=46601
    Range : 0 ... 46600 =      0.000 ...   198.298 secs
Ready.
Effective window size : 1.089 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47001
    Range : 0 ... 47000 =      0.000 ...   200.000 secs
Ready.
Effective window size : 1.089 (s)
Creating RawArray with float64 data, n_channels=62, n_times=47601
    Range : 0 ... 47600 =      0.000 ...   202.553 secs
Ready.
Effective window size : 1.089 (s)
Creating RawArray with

In [7]:
subject_psd_data.shape

(30, 310)

In [11]:
subject_psd_data.head()

,channel_0_delta,channel_0_theta,channel_0_alpha,channel_0_beta,channel_0_gamma,channel_1_delta,channel_1_theta,channel_1_alpha,channel_1_beta,channel_1_gamma,...,channel_60_delta,channel_60_theta,channel_60_alpha,channel_60_beta,channel_60_gamma,channel_61_delta,channel_61_theta,channel_61_alpha,channel_61_beta,channel_61_gamma
0,160.319669,147.985806,143.611531,60.759257,51.729120,109.304214,39.511108,21.781194,15.794086,12.803319,...,0.157912,0.097908,0.127913,19236.751966,0.481712,0.670039,0.534511,0.265850,0.216600,0.205642
1,229.988270,214.932589,207.687166,88.566063,72.826902,373.514722,60.181025,32.351322,22.989578,18.434479,...,0.188477,0.106531,0.156842,119860.817192,0.506793,1.073040,0.762018,0.323736,0.303457,0.283959
2,162.592514,156.373772,156.502053,62.542880,55.725939,314.963529,43.304917,22.900683,16.491393,14.132715,...,0.130108,0.084601,0.114747,122061.492223,0.287378,0.591181,0.441504,0.249960,0.242571,0.257963
3,174.787534,164.065440,156.004183,68.002411,54.313884,563.765332,46.938473,24.762149,18.046400,14.745354,...,0.129453,0.075443,0.095708,126828.602232,0.309133,0.588686,0.434834,0.222883,0.159525,0.156558
4,139.697263,130.769811,126.011234,53.897967,45.209369,469.930254,40.352609,19.316803,14.300490,11.852457,...,0.148021,0.083806,0.104224,134578.979642,0.347898,0.800523,0.566521,0.260677,0.180274,0.183974
